In [1]:
import tensorflow as tf
print(tf.__version__)

In [2]:
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [3]:
#Reduce the size of the images to reduce the model size. 128x128 = 812MB
#Rescale the image sizes to between 0 and 1
train_img_datagen = ImageDataGenerator(rescale=1./255)
val_img_datagen = ImageDataGenerator(rescale=1./255)

train_set = train_img_datagen.flow_from_directory(
    '../input/food-train/Food_train',
    target_size=(32,32),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical')

val_set = val_img_datagen.flow_from_directory(
    '../input/food-test/Food_test',
    target_size=(32,32),
    batch_size=32,
    color_mode='rgb',
    class_mode='categorical')

In [4]:
model = tf.keras.Sequential()
model.add(Conv2D(filters=32, kernel_size=(3), input_shape=(32,32,3)))
model.output_shape

In [5]:
model.add(Flatten(input_shape=(32, 32)))
model.output_shape

In [6]:
model.add(Dense(128, activation='linear'))
model.output_shape
model.add(Dense(2, activation='softmax'))
model.output_shape

In [7]:
#model = Sequential([
#    Conv2D(filters=32, kernel_size=1, input_shape=(32,32,3)),
#    Flatten(input_shape=(32, 32)), #flatten --> 3D to 2D
#    Dense(16, activation='relu'),
#    Dense(2, activation='softmax')
#])

In [8]:
#Use BinaryCrossentropy for classification of 0 and 1
#https://www.tensorflow.org/api_docs/python/tf/keras/losses/BinaryCrossentropy
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [9]:
model_info = model.fit(train_set,batch_size=32, validation_data= val_set, epochs=5, shuffle=True)

model.summary()

In [10]:
import matplotlib.pyplot as plt

# list all data in history
print(model_info.history.keys())

# summarize history for accuracy
plt.plot(model_info.history['accuracy'])
plt.plot(model_info.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(model_info.history['loss'])
plt.plot(model_info.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [11]:
model.save('CharKwayTeowNasiLemakRGBWorking.h5')

In [12]:
from keras.models import load_model
model = load_model('CharKwayTeowNasiLemakRGBWorking.h5')

In [13]:
class_dict = {'CharKwayTeow': 0,
 'NasiLemak': 1}

class_names = list(class_dict.keys())

In [14]:
from keras.models import load_model
from keras.preprocessing import image
import matplotlib.pyplot as plt
import numpy as np

# dimensions of our images
img_width, img_height = 32, 32
images_dir = "../input/food-score/food_score"

In [18]:
# predicting single image
image_path = "../input/food-score/Nasilemak5.jpg"
new_img = image.load_img(image_path, target_size=(img_width, img_height))
img = image.img_to_array(new_img)
img = np.expand_dims(img, axis=0)
img = img/255

img_class = model.predict_classes(img) #returns ndim np_array
print(img_class)
img_class_index = img_class.item() #extracting value(s)
print(img_class_index)
classname = class_names[img_class_index]#[Charkwayteoew, Nsailamek]

img_prob = model.predict_proba(img) #returns numpy array of class probabilities
print(img_prob);
prediction_prob = img_prob.max()

pred_dict = {"Class":classname, "Probability":prediction_prob}
print(pred_dict)

#ploting image with predicted class name        
plt.figure(figsize = (4,4))
plt.imshow(new_img)
plt.axis('off')
plt.title(classname)
plt.show()